# setup

In [1]:
!pip install yt_dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 9.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import glob
import os
import pandas as pd
import yt_dlp

path = "/content/drive/MyDrive/uraradi_transcript/"

# identify existing mp3/transcript

In [4]:
mp3_list = []
for sounds_file in glob.glob(path+"sounds/*.mp3"):
    filename = sounds_file.split("/")[-1].split(".")[0]
    mp3_list.append(filename)

csv_list = []
for transcripts_file in glob.glob(path+"transcripts/*.csv"):
    filename = transcripts_file.split("/")[-1].split(".")[0]
    csv_list.append(filename)

# identify target radio

In [5]:
playlist = pd.read_csv(path + "playlist_裏ラジオウルナイト.csv")
playlist["flag"] = playlist["date"].apply(lambda date: (date not in mp3_list)&(date not in csv_list))
target_radios = playlist[playlist["flag"]==True][["url", "date"]].reset_index(drop=True).copy()
target_radios["finished"] = False
target_radios

,url,date,finished
0,https://www.youtube.com/watch?v=YCvPUXbebzg,2023-03-10,False


In [8]:
ydl_opts = {
    "paths": {"home": path+"sounds"},
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }]
}
for _, radio in target_radios.iterrows():
    if target_radios.at[0, "finished"] == False:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(radio["url"])
            info = ydl.extract_info(radio["url"], download=False)
        old_file = path + "sounds/" + info["title"].replace("/", "⧸") + " [" + info["id"] + "]" + ".mp3"
        new_file = path + "sounds/" + radio["date"] + ".mp3"
        if os.path.exists(old_file):
            os.rename(old_file, new_file)
            print("file renamed!")
        target_radios["finished"] == True

[youtube] Extracting URL: https://www.youtube.com/watch?v=YCvPUXbebzg
[youtube] YCvPUXbebzg: Downloading webpage
[youtube] YCvPUXbebzg: Downloading android player API JSON
[info] YCvPUXbebzg: Downloading 1 format(s): 140
[dashsegments] Total fragments: 5
[download] Destination: /content/drive/MyDrive/uraradi_transcript/sounds/ラジオネームChatGPT｜#74.5 裏ラジオウルナイト｜大浦るかこ ⧸⧸ あにまーれ [YCvPUXbebzg].m4a
[download] 100% of   42.70MiB in 00:00:07 at 5.43MiB/s               
[FixupM4a] Correcting container of "/content/drive/MyDrive/uraradi_transcript/sounds/ラジオネームChatGPT｜#74.5 裏ラジオウルナイト｜大浦るかこ ⧸⧸ あにまーれ [YCvPUXbebzg].m4a"
[ExtractAudio] Destination: /content/drive/MyDrive/uraradi_transcript/sounds/ラジオネームChatGPT｜#74.5 裏ラジオウルナイト｜大浦るかこ ⧸⧸ あにまーれ [YCvPUXbebzg].mp3
Deleting original file /content/drive/MyDrive/uraradi_transcript/sounds/ラジオネームChatGPT｜#74.5 裏ラジオウルナイト｜大浦るかこ ⧸⧸ あにまーれ [YCvPUXbebzg].m4a (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=YCvPUXbebzg
[youtube] YCvPUXbebzg: Down